In [1]:
using Revise

# A tutorial for the package ```ChebyshevFiltering```.

In this notebook we will provide a tutorial on how to use the the package ```ChebyshevFiltering``` for sparse diagonalization of Hermitian matrices (both real and complex).

Let us first load the package

In [2]:
using ChebyshevFiltering
include("/home/dario/Dropbox/delocalization_and_growth/revision_chaos_paper_code/QMB_module.jl")
using .ImpurityUtilities

and now let's create an SYK-like Hamiltonian. We also renormalize its spectrum to lie in the interval $\left[-1 , \, 1 \right]$.

In [3]:
const number_of_majorana = 24
majorana_matrices = majorana_matrices_chiral(number_of_majorana)
hamiltonian_matrix, edges_graph, weights_graph = hamiltonian_chiral(number_of_majorana, majorana_matrices, 4, 1.0)
renormalization_hamiltonian!(hamiltonian_matrix)

2048×2048 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 69632 stored entries:
⎡⢿⣷⣿⣶⡾⠳⣄⠘⣶⡞⠃⠀⠀⠀⠀⠀⠀⢸⣾⡿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⣽⣾⡷⎤
⎢⢻⣿⣿⣿⣽⣦⣬⡿⣏⠙⠂⠀⠀⠀⠀⣦⣶⣾⠛⠙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢐⣦⣾⡿⣟⠙⎥
⎢⢾⡋⠳⣿⣿⣿⣟⣦⣈⣳⡄⠀⢠⣠⣤⡿⠟⠦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣄⠀⢤⣠⣼⠿⠟⠆⠈⠳⎥
⎢⣀⠙⣦⡿⠻⣽⣿⣿⣿⢧⡀⣀⣸⣿⢯⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡈⣳⣿⣿⢯⠀⠀⠀⠀⠀⎥
⎢⣸⠿⣏⠙⢦⣸⠿⣟⣿⣿⣿⣿⡛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢐⣽⢿⡋⠳⣄⠀⠀⠀⠀⠀⎥
⎢⠉⠀⠈⠀⠀⠉⠀⢨⣿⣿⣿⣿⣿⣶⡟⠳⣄⢘⣶⡟⠀⠀⠀⠀⠈⠳⣄⣽⣾⡿⠁⠁⠁⠀⠈⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⣲⣶⣾⠛⠈⢻⣿⢿⣷⣿⣦⣼⠿⣇⠙⠀⠀⠀⠀⢐⣦⣾⡿⣟⠙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠠⣤⣤⡿⠏⠳⠀⠀⢿⡉⠻⣿⣿⣿⣽⣦⣈⣷⣄⠀⢤⣠⣬⠻⠟⠆⠈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⣀⣸⣿⠻⡅⠀⠀⠀⠀⣀⢙⣶⡟⠳⣿⣿⣿⣟⣧⡈⣳⣿⣿⢯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣾⡿⣟⠀⠀⠀⠀⠀⠀⠀⣼⠿⣍⠙⢦⣼⠿⣽⣿⣿⣿⣿⡋⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⣨⣿⣿⣿⣿⣟⣶⡟⠳⣄⣙⣶⡟⠀⠀⠀⠀⠀⠀⠀⣽⣾⡿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣳⣿⣿⢯⡈⢻⣽⣿⣿⣿⢦⣼⠿⣅⠉⠀⠀⠀⠀⢘⣦⣿⡏⠉⠉⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡀⠰⣴⣦⡛⠋⠓⠀⠙⢿⡉⠻⣟⣿⣿⣿⣦⣈⣷⠀⠀⢦⣰⣾⠛⠛⠂⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣄⣽⣾⡿⠻⠅⠀⠀⠀⠀⣄⢹⣶⡟⠻⣿⢿⣷⣿⣧⡀⣤⡿⠿⠯⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⡀⠀⢀⢀⢀⣾⡿⣟⠙⢦⡀⠀⠀⠀⠀⣼⠿⡅⠙⢦⣼⠿⣿⣿⣿⣿⣿⡃⠀⣀⠀⠀⡀⠀⣀⎥
⎢⠀⠀⠀⠀⠀⠙⢦⣨⣷⣟⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣬⣿⣿⣿⣿⣽⣶⡏⠳⣄⣹⣶⡏⎥
⎢⠀⠀⠀⠀⠀⣳⣿⣿⢯⡈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⣳⣿⡏⠉⠈⢳⣿⣿⣿⣟⣦⣾⠻⣄⠉⎥
⎢⢦⡀⠰⣴⣶⡟⠋⠓⠀⠙⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠲⣴⣾⠛⠋⠃⠀⠘⢯⡉⠻⣽⣿⣿⣿⢦⣨⡷⎥
⎢⣄⣽⣾⡿⠻⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣄⣤⡿⠿⠻⠀⠀⠀⠀⠠⣄⣹⣾⡛⠻⣟⣿⣿⣿⣧⎥
⎣⢾⡿⣟⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⡿⡇⠀⠀⠀⠀⠀⠀⢠⡼⠿⡄⠙⢦⡾⠿⣿⢿⣷⎦

## The estimate of the density of states via Kernel Polynomial Method.

We now test how works the estimate of the density of states, computed by means of the Kernel Polynomial Method.

We start by fixing the relevant parameters, and the window of the spectrum where we want to estimate the density

In [5]:
max_degree_KPM = 150
stochastic_dimension_KPM = 30
lambda_min = -0.1
lambda_max = 0.1

"log_try.txt"

and now we can compute the approximated density and compute the number of expected states

In [6]:
approximated_density = ChebyshevFiltering.KPM_density(hamiltonian_matrix, max_degree_KPM, stochastic_dimension_KPM)
x_values = lambda_min:((lambda_max - lambda_min) / 200):lambda_max
y_values = [approximated_density(x) for x in x_values]

using NumericalIntegration
expected_value = NumericalIntegration.integrate(x_values, y_values)

98.26430615829746

which we compare against the actual value:

In [7]:
using LinearAlgebra
real_number = length([value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

98

We see that the agreement is excellent.

## The eigenvalues via ```eigen_cheb```

Now we can test the Chebyshev filtering main function: the computation of the eigenvalues in a given interval by means of ```eigen_cheb```.

We first define the parameters:

In [4]:
log_path = "/home/dario/Dropbox/delocalization_and_growth_data/log_try/"
log_file_name = "log_try.txt"
lambda_min = -0.1
lambda_max = 0.1

0.1

then we compute the eigenvalues/eigenvectors in the interval chosen

In [5]:
converged_target_values, converged_target_vectors = eigen_cheb(hamiltonian_matrix, log_path, log_file_name, lambda_min, lambda_max)

([-0.09960309392020116, -0.09885425057729062, -0.09862879382636705, -0.0983482843202829, -0.09792552523396315, -0.09757938238699777, -0.09683760562332372, -0.09626659245060926, -0.09527731765837714, -0.09495398140872739  …  0.09551990604377356, 0.09621852557401507, 0.09694445744781316, 0.0970868155895651, 0.09748330254860936, 0.09771165941954614, 0.09829018975769183, 0.09868470226616975, 0.09897777220737634, 0.09962516913506429], Vector{ComplexF64}[[-0.0015593015380220736 - 0.01015424718722303im, 0.014122345797117224 - 0.021570906840377914im, 0.012253991161351828 + 0.0186761174851864im, 0.02925034802451029 + 0.003753644610090578im, 0.005196955400664336 + 0.00424038862522062im, -0.024010802692390407 + 0.0002641982295828624im, -0.002892501715869857 - 0.019472740519091484im, -0.009559691312289738 + 0.001241661442271542im, 0.014942909722378996 - 0.003638672616724456im, 0.013241093878207096 + 0.010835663521415191im  …  0.0021059071049066612 + 0.028345680591866235im, -0.018764752130325732 - 

that we compare against the actual result

In [6]:
using LinearAlgebra
isapprox(converged_target_values, [value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

true

we see that it works